In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
!pip install -q findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

#spark = SparkSession.builder.master("local[*]").getOrCreate()
print(f'num executors: {sc.getConf().get("spark.executor.instances")}')

num executors: None


In [ ]:
!pip install -q mmh3

     |████████████████████████████████| 50 kB 2.9 MB/s 


In [ ]:
import time
import math
import numpy as np
import random
import mmh3
import pandas as pd
import tqdm
import pickle
import matplotlib.pyplot as plt
from pyspark.sql.functions import col

from sklearn.metrics import roc_auc_score, average_precision_score
from pyspark.mllib.evaluation import BinaryClassificationMetrics

np.set_printoptions(threshold=50)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Import Data


In [ ]:
print("Running xStream")
print(sc.defaultParallelism)

Running xStream
2


## Exec: Setup

In [ ]:
projection=True
isMutable=True #set to True if mixed-attributes
projdim = 30 #K
nchains = 50
depth = 15

r = 3
w = 50

numPartitions = 64
nthreads_fit = 128
nthreads_score = 128
samplerate =  1 

In [ ]:
part = 99
sys_config = 7
model_file = "model_" + str(nchains) + "_" + str(depth) + ".pkl"
#output_file = "output_score_" + str(nchains) + "_" + str(depth) + "_" + str(int(samplerate * 100)) + "_" + str(sys_config) + ".txt"
output_file = "output_f1.txt"
score_folder = "score_" + str(nchains) + "_" + str(depth) + "_" + str(int(samplerate * 100)) + "_" + str(sys_config) +".csv"


In [ ]:
# splitting file
split = False
if split:
    df = pd.read_csv('/ocean/projects/ccr190032p/shared/osm_data/simple-gps-combined-1-with-labels.txt', sep=',', header=None, error_bad_lines=False)
    count = df.shape[0]
    split_size = 10
    df_1 = df.iloc[:int(count/split_size),:]
    df_2 = df.iloc[int(count/split_size):int(2*count/split_size),:]
    df_3 = df.iloc[int(2*count/split_size):int(3*count/split_size),:]
    df_4 = df.iloc[int(3*count/split_size):int(4*count/split_size),:]
    df_5 = df.iloc[int(4*count/split_size):int(5*count/split_size),:]
    df_6 = df.iloc[int(5*count/split_size):int(6*count/split_size),:]
    df_7 = df.iloc[int(6*count/split_size):int(7*count/split_size),:]
    df_8 = df.iloc[int(7*count/split_size):int(8*count/split_size),:]
    df_9 = df.iloc[int(8*count/split_size):int(9*count/split_size),:]
    df_10 = df.iloc[int(9*count/split_size):,:]
    
    df_1.to_csv("/ocean/projects/cie170025p/shared/osm_10/osm_all_1.txt", header=None, index=None)

In [ ]:
test_big = True
if test_big:
    start = time.time()
    #myDF = spark.read.options(inferSchema='True').csv("synthData-10m.csv")
    #myDF = spark.read.options(inferSchema='True').csv("/ocean/projects/ccr190032p/shared/synthData-big/ionosphere_5m.X.csv")
    #myDF = spark.read.options(inferSchema='True').csv("test-data.csv")
    #myDF = spark.read.options(inferSchema='True').csv("synDataNoisy_all.csv")
    #myDF = spark.read.options(inferSchema='True').csv("/ocean/projects/ccr190032p/shared/synthData-big/gisette_40k_3.all.csv")
    #myDF = spark.read.options(inferSchema='True').csv("/ocean/projects/ccr190032p/shared/osm_data/simple-gps-combined-1-with-labels.txt")
    #myDF = spark.read.options(inferSchema='True').csv("/ocean/projects/ccr190032p/shared/Spam_URL/SpamURL_100_labels.csv")
 
    myDF = spark.read.options(inferSchema='True').csv("/content/drive/My\ Drive/ColabNotebooks/data/test-data-x.csv")
    #myDF = spark.read.options(inferSchema='True').csv("/content/drive/My\ Drive/ColabNotebooks/data/test-data-ohe-x.csv")
    
    myDF = myDF.na.fill('').na.fill(0)
    
    end = time.time()
    count = myDF.count()
    print(myDF.take(5))
    print("Read time = %.3fs" % (end - start))
    print(myDF.rdd.getNumPartitions())
    print((count, len(myDF.columns)))
    

[Row(_c0='Male', _c1='Bulgaria', _c2=34, _c3=100, _c4=0), Row(_c0='Male', _c1='Bulgaria', _c2=36, _c3=140, _c4=0), Row(_c0='Male', _c1='India', _c2=32, _c3=120, _c4=0), Row(_c0='Female', _c1='Turkey', _c2=33, _c3=20, _c4=1), Row(_c0='Male', _c1='Germany', _c2=45, _c3=75, _c4=0)]
Read time = 10.464s
1
(7, 5)


In [ ]:
myDF.printSchema()
#myDF.dtypes

root
 |-- _c0: string (nullable = false)
 |-- _c1: string (nullable = false)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)



In [ ]:
repartition = True
if repartition:
    myDF = myDF.repartition(numPartitions)
    print("Repartition:", myDF.rdd.getNumPartitions())
print((myDF.count(), len(myDF.columns)))
    
#y = np.array(myDF.select(myDF.columns[-1]).collect())
#y=y.reshape(-1)
#print(y.shape)
#myDF = myDF.select(myDF.columns[0:-1])
#myDF_new = myDF.rdd.map(lambda x: (list(x[0:-1]), x[-1]))
#myDF_new.count()
all_columns = myDF.columns[:-1]

Repartition: 64
(7, 5)


In [ ]:
downsize = False
factor = 0.0000001
if downsize:
    myDF = myDF.limit(int(count * factor))

##Min-Max Normalize Numerical Features Prior to Projection

In [ ]:
from pyspark.sql.functions import max as sparkMax
from pyspark.sql.functions import min as sparkMin

def min_max_normalize(df):
    feature_names = all_columns
    print(feature_names)
    # getting the max and min for all columns
    max_df = df.select(*[sparkMax(col(c)).alias(c) for c in feature_names])
    all_max = max_df.toPandas().to_numpy()[0]
    min_df = df.select(*[sparkMin(col(c)).alias(c) for c in feature_names])
    all_min = min_df.toPandas().to_numpy()[0]

    #print(all_max)
    for i in range(len(feature_names)): # <-- THREADS?
        if ((df.dtypes[i][1]) == 'string'):
            continue
        #min_val = df.select(feature_names[i]).rdd.reduce(lambda a, b: a if a < b else b)[0]
        #max_val = df.select(feature_names[i]).rdd.reduce(lambda a, b: a if a > b else b)[0]
        #min_val = df.agg({feature_names[i]: "min"}).collect()[0][0]
        #max_val = df.agg({feature_names[i]: "max"}).collect()[0][0]
        min_val = all_min[i] #.item()
        max_val = all_max[i] #.item()
        
        if (min_val == 0 and max_val == 1):
           continue
        print(feature_names[i])
        df = df.withColumn(feature_names[i],(col(feature_names[i]) - min_val) / (max_val - min_val))
        # min-max takes >50x time compared to normalize
        # IS THIS <withColumn> LOCAL?  
    return df # <-- IS THIS RETURNING/COLLECTING TO THE DRIVER?

In [ ]:
# all_outputs = []
# min_max = True
# if min_max:
#     print("Min-max normalize")
#     all_outputs.append("Min-max normalize")
#     #start = time.time()
#     normDF = min_max_normalize(myDF)
#     #normDF.show()
#     #end = time.time()
#     #print("Time = %.3fs" % (end-start))
#     #all_outputs.append("Time = %.3fs" % (end-start))
# else:
#     normDF = myDF

In [ ]:
class StreamhashProjection:

    def __init__(self, n_components, feature_names=None, density=1/3.0, random_state=None, mutable=False, sc=None):
        self.keys = np.arange(0, n_components, 1)
        self.constant = np.sqrt(1./density)/np.sqrt(n_components)
        self.density = density
        self.n_components = n_components
        self.mutable=mutable
        random.seed(random_state)
        
        input_dim = len(feature_names)
        self.R = sc.broadcast(np.array([[self._hash_string(k, f) # shape: (len(self.keys), input_dim)
                       for f in feature_names]
                       for k in self.keys]))

    def fit_transform(self, X, feature_names=None): 

        R = self.R.value  
          
        if self.mutable:
            #nsamples = 1      # X.shape[0]
            ndim = len(X)     # X.shape[1]
            if feature_names is None:
                feature_names = [str(i) for i in range(ndim)]

            types = [type(X[i]) == str for i in range(ndim)] #!!! [0][i] not [i]

            feature_names = [feature_names[i]+'.'+X[i] if types[i] else feature_names[i] for i in range(ndim)]
            X = [1 if types[i] else X[i] for i in range(ndim)]

            
            for i in range(ndim):
                if types[i]:
                    f = feature_names[i]
                    R[:,i] = np.array([self._hash_string(k, f) for k in self.keys])
                
            Y = np.dot(X, R.T)
        else:
            Y = np.dot(X, self.R.T)
        
        return Y

    def explain_transform(self, X, feature_names=None): 

        R = self.R.value  
          
        if self.mutable:
            #nsamples = 1      # X.shape[0]
            ndim = len(X)     # X.shape[1]
            if feature_names is None:
                feature_names = [str(i) for i in range(ndim)]

            types = [type(X[i]) == str for i in range(ndim)] #!!! [0][i] not [i]

            feature_names = [feature_names[i]+'.'+X[i] if types[i] else feature_names[i] for i in range(ndim)]
            #X = [1 if types[i] else X[i] for i in range(ndim)]

            
            for i in range(ndim):
                if types[i]:
                    f = feature_names[i]
                    R[:,i] = np.array([self._hash_string(k, f) for k in self.keys])
                
            #Y = np.dot(X, R.T)
        #else:
            #Y = np.dot(X, self.R.T)
        
        return R
        
    def explain_transform_ohe(self, allX, feature_names=None): 

        #numunique is a ndim-dimensional vector containing number of unique values per feature
     
        R = self.R.value

        if self.mutable:
            if feature_names is None:
                feature_names = [str(i) for i in range(ndim)]

            ndim = len(feature_names)

            X = allX.take(1)
            #print(X)
            types = [type(X[0][i]) == str for i in range(ndim)]
            #print(types)
            # find total # of features after OHE
            num_unique = np.zeros((ndim,1))
            for i in range(ndim):
                if types[i]:
                  num_unique[i] = allX.select(allX.columns[i]).distinct().count()
                else:
                  num_unique[i] = 1

            #print(num_unique)

            totalAfterOHE = num_unique.sum()
            
            R_ohe = np.zeros((self.n_components, int(totalAfterOHE))) 
            
            #feature_names = [feature_names[i]+'.'+X[i] if types[i] else feature_names[i] for i in range(ndim)]
     
            ind = 0
            for i in range(ndim): # for each original feature
                if types[i]:
                    # find all distinct values
                    distinctDF = allX.select(allX.columns[i]).distinct()
                    outrdd = distinctDF.rdd.map(lambda x: (x[0], np.array([self._hash_string(k, feature_names[i]+'.'+x[0]) for k in self.keys])))
                    for v in outrdd.collect():
                        R_ohe[:,ind] = v[1]
                        ind = ind+1
                else:
                  f = feature_names[i]
                  R_ohe[:,ind] = np.array([self._hash_string(k, f) for k in self.keys])
                  ind = ind+1 # increment by 1 as just 1 ohe feature

            return R_ohe
            
        else:
            return R


    def _hash_string(self, k, s):
        hash_value = int(mmh3.hash(s, signed=False, seed=k))/(2.0**32-1)
        den = self.density
        if hash_value <= den/2.0:
            return self.constant # -1 *
        elif hash_value <= den:
            return self.constant
        else:
            return 0

In [ ]:
def projectDF(df, projdim, isMutable, sc=None):
    feature_names = all_columns
    #for i in range(len(feature_names)):
    #  print(feature_names[i])
    projector = StreamhashProjection(n_components=projdim, 
                                     feature_names=feature_names,
                                     density=1/3.0, 
                                     random_state=42, 
                                     mutable=isMutable,
                                     sc=sc)
    projectedDF = df.rdd.map(lambda x: projector.fit_transform(list(x[0:-1]),feature_names).tolist() + [x[-1]]).toDF()  

    return projectedDF

In [ ]:
def feature_range(df):
    features = df.columns[:-1]
    """a = time.time()
    max_df = df.select(*[sparkMax(col(c)).alias(c) for c in features])
    all_max = max_df.toPandas().to_numpy()[0]
    a2 = time.time()
    print(a2-a)
    min_df = df.select(*[sparkMin(col(c)).alias(c) for c in features])
    all_min = min_df.toPandas().to_numpy()[0]
    detamax = all_max - all_min
    b = time.time()
    print(b-a2)"""
    deltamax = np.zeros(len(features), dtype=np.float)  
    for f in range(len(features)):
        #deltamax = [(df.select(f).rdd.reduce(lambda a, b: a if a > b else b)[0] - df.select(f).rdd.reduce(lambda a, b: a if a < b else b)[0])/2.0 for f in features]
        #fRDD = df.rdd.map(lambda x: x[f])
        #max_val_f = fRDD.reduce(lambda a, b: a if a > b else b)
        #min_val_f = fRDD.reduce(lambda a, b: a if a < b else b)
        #deltamax[f] = max_val_f - min_val_f
        #deltamax[f] = df.select(f).rdd.max()[0][0] - df.select(f).rdd.min()[0][0]
        deltamax[f] = (df.agg({features[f]: "max"}).collect()[0][0] - df.agg({features[f]: "min"}).collect()[0][0])/2
    c = time.time()
    #print(c-b)
    return deltamax

## Project the Data

In [ ]:
    # print("Projection")
    # all_outputs.append("Projection")
    # start = time.time()
    # if projection:
    #     pDF = projectDF(normDF, projdim, isMutable, sc=sc)
    # else:
    #     pDF = normDF
      
    # deltamax = feature_range(pDF)
    # #print(deltamax)
    # #### convert into pair rdd ####
    # pDF_pair = pDF.rdd.map(lambda x: (list(x[0:-1]), x[-1]))
    # pDF_pair.cache()
    # end = time.time()
    # print("Time = %.3fs" % (end-start))
    # all_outputs.append("Time = %.3fs" % (end-start))

In [ ]:
np.random.seed(42)
from array import array

class CMS:
    def __init__(self, r, w):
        self.r = r
        self.w = w
    
        upper_bound = 2147483647
        step = upper_bound / (r-1)
        manyranges = [(i*step, step*(i+1)-1) for i in range(r-1)]
        self.mask = array('L', (np.random.randint(low, high) for low, high in manyranges))

    def findAllRowCols(self, myL):          
        h = hash(str(myL)) % self.w

        result = []
        result.append( (1, h) )
        row=2
        for m in self.mask:
            result.append( (row, (h ^ m) % self.w ))
            row+=1
        return result

    def allCols(self, X):
        h = hash(str(X)) % self.w

        result = []          
        result.append( ((1, h), 1) )
        row=2
        for m in self.mask:
            result.append( ((row, (h ^ m) % self.w), 1) )
            row+=1
        return result

cms = CMS(r=r, w=w)

In [ ]:
class Chain:
  
    def __init__(self, depth, deltamax):   # depth = 6
        k = len(deltamax)
        self.deltamax = deltamax # feature ranges
        self.depth = depth
        self.fs = [np.random.randint(0, k) for d in range(depth)]
        print(self.fs)
        ### self.cmsketches = [{}] * depth #self.cmsketches = [None] * depth
        #???????# does this become a common data structure
        self.shift = np.random.rand(k) * deltamax

    # input X is a **single** point 
    def fit(self, X, verbose=False): #!!! all depths
        
        # initialize cmsketch tables
        prebin = np.zeros(len(X), dtype=np.float)
        depthcount = np.zeros(len(self.deltamax), dtype=np.int)
        
        ls = [None] * depth
        for d in range(self.depth):
          f = self.fs[d] #split feature at depth d
          depthcount[f] += 1

          if depthcount[f] == 1:
              prebin[f] = (X[f] + self.shift[f])/self.deltamax[f] 
          else:
              prebin[f] = 2.0*prebin[f] - self.shift[f]/self.deltamax[f]
          
          
          ls[d] = tuple(np.floor(prebin).astype(np.int)) ## MAYBE JUST RETURN l's and then do <unique,count>
          #if not l in cmsketch:
          #    cmsketch[l] = 0
          #cmsketch[l] += 1

        #self.cmsketches[depth] = cmsketch

        return ls #<-- contains the integer bind-id vector at each level for the *single* X


    def bincount_score(self, X, cmsketches):
        # calculate the score at every depth

        scores = np.zeros(self.depth)
       
        prebin = np.zeros(len(X), dtype=np.float)
        depthcount = np.zeros(len(self.deltamax), dtype=np.int)
  
        for d in range(self.depth):
            f = self.fs[d] 
            depthcount[f] += 1
            if depthcount[f] == 1:
                prebin[f] = (X[f] + self.shift[f])/self.deltamax[f]
            else:
                prebin[f] = 2.0*prebin[f] - self.shift[f]/self.deltamax[f]

            cmsketch = cmsketches[d]
            l = tuple(np.floor(prebin).astype(np.int))

            rowcols = cms.findAllRowCols(l)
            counts = [cmsketch.get(rowcol, 0) for rowcol in rowcols]
            #print(d, counts)
            scores[d] = min(counts)
            
        
        # scale score logarithmically to avoid overflow:
        # score = min_d [ log2(bincount x 2^d) = log2(bincount) + d ]
        depths = np.array([d for d in range(1, self.depth+1)])
        scores = np.log2(1.0 + scores) + depths 
        return np.min(scores)

    def bincount_score_exp(self, X, cmsketches):
        # calculate the score at every depth

        scoresAllDepths = np.zeros(self.depth)
        
        prebin = np.zeros(len(X), dtype=np.float)
        depthcount = np.zeros(len(self.deltamax), dtype=np.int)
  
        for d in range(self.depth):
            f = self.fs[d] 
            depthcount[f] += 1
            if depthcount[f] == 1:
                prebin[f] = (X[f] + self.shift[f])/self.deltamax[f]
            else:
                prebin[f] = 2.0*prebin[f] - self.shift[f]/self.deltamax[f]

            cmsketch = cmsketches[d]
            l = tuple(np.floor(prebin).astype(np.int))

            rowcols = cms.findAllRowCols(l)
            counts = [cmsketch.get(rowcol, 0) for rowcol in rowcols]
            #print(d, counts)
            scoresAllDepths[d] = min(counts)
            
        #print(scores, scores_2)
        
        # scale score logarithmically to avoid overflow:
        # score = min_d [ log2(bincount x 2^d) = log2(bincount) + d ]
        depths = np.array([d for d in range(1, self.depth+1)])
        scoresAllDepths = np.log2(1.0 + scoresAllDepths) + depths 
        return scoresAllDepths

In [ ]:
from multiprocessing.pool import ThreadPool
from functools import reduce
from operator import add

class Chains:
    def __init__(self, deltamax, nchains=1, depth=2, nthreads_fit=1, nthreads_score=128, samplerate=0.1): #k=3 , seed=42
        self.nchains = nchains
        self.depth = depth
        self.nthreads_fit = nthreads_fit
        self.nthreads_score = nthreads_score
        self.samplerate = samplerate
        self.deltamax = deltamax
        self.chains_cmsketches = [None] * nchains # will be made a boardcast variable after fitting
        self.chains = [None] * nchains
        #self.projector = StreamhashProjection(n_components=k, density=1/3.0, random_state=seed)  # StreamhashProjection(n_components=k, density=1/3.0, random_state=seed)

    def fitparallel(self, allX):   
        pool = ThreadPool(self.nthreads_fit)

        def fitone(cindex):     

            # create i'th Chain
            c = Chain(depth=self.depth, deltamax=self.deltamax)
            self.chains[cindex] = c

            binvecsRDD = allX.sample(False,self.samplerate, cindex*100).map(lambda x: c.fit(x[0])) # <-- inject random sampling

            cmsketches = [{}] * self.depth
            for d in range(self.depth):
                cmsketches[d] = binvecsRDD.flatMap(lambda x: cms.allCols(x[d])).reduceByKey(lambda x,y:x+y).collectAsMap()
            self.chains_cmsketches[cindex] = cmsketches
            
            return [cindex]
        
        parameters = list(range(self.nchains))
        pool.map(lambda chainindex: fitone(chainindex), parameters)

        return self
    

    def fit(self, allX):     
        for i in tqdm.tqdm(range(self.nchains), desc='Fitting...'): # <-- THREADS?
            # create i'th Chain
            c = Chain(depth=self.depth, deltamax=self.deltamax)
            self.chains[cindex] = c

            #binvecsRDD = allX.rdd.map(lambda x: c.fit(x)) # <-- inject random sampling
            binvecsRDD = allX.rdd.sample(False,self.samplerate,i*100).map(lambda x: c.fit(x)) # <-- inject random sampling
            #binvecsRDD = allX.rdd.filter(inside).map(lambda x: c.fit(x)) # <-- inject random sampling

            cmsketches = [{}] * self.depth
            for d in range(self.depth):
                cmsketches[d] = binvecsRDD.map(lambda x: (x[d], 1)).reduceByKey(lambda x,y:x+y).collectAsMap()
            self.chains_cmsketches[cindex] = cmsketches
        
        return self

    def scoreparallel_test(self, allX):
        time_a = time.time()
        pool = ThreadPool(self.nthreads_score)

        def scoreone(cindex):     
            chain = self.chains[cindex]
            scores_rdd = allX.map(lambda x:(chain.bincount_score(x[0], self.chains_cmsketches[cindex]), x[1]))
            return scores_rdd.zipWithIndex().map(lambda x:(x[1], x[0])) # (index, (score, label))
        
        parameters = list(range(self.nchains))
        all_score_rdds = pool.map(lambda chainindex: scoreone(chainindex), parameters)
        assert(len(all_score_rdds) == self.nchains)
        time_b = time.time()
        print("Score time = ", time_b-time_a)
        
        sum_rdd = all_score_rdds[0]
        for i in range(1, self.nchains):
            score_rdd = all_score_rdds[i]
            sum_rdd = sum_rdd + score_rdd # concatenate
            
        totalscore_rdd = sum_rdd.reduceByKey(lambda x,y:(x[0]+y[0], x[1])) # key is index, value is (totalscore,label)
        time_c = time.time()
        print("Combine time new = ", time_c-time_b)

        return totalscore_rdd.map(lambda x: ( x[0], (float(-x[1][0]/self.nchains), float(x[1][1])) ) ) #  key is index, value is (score,label)
 

    def score(self, X):
        #scores = np.zeros(allX.count())
        #chain = self.chains[0]
        totalscore = 0;
        #scoresRDD = allX.rdd.map(lambda x: chain.bincount_score(x, self.chains_cmsketches[0]))
        for i in range(self.nchains): # <-- THREADS?
            chain = self.chains[i]
            totalscore += chain.bincount_score(X, self.chains_cmsketches[i])
        return -totalscore/float(self.nchains)


## Exec: Min-Max Normalize

In [ ]:
    all_outputs = []
    min_max = True
    if min_max:
        print("Min-max normalize")
        all_outputs.append("Min-max normalize")
        start = time.time()
        normDF = min_max_normalize(myDF) #100+ columns
        normDF.show()
        end = time.time()
        print("Time = %.3fs" % (end-start))
        all_outputs.append("Time = %.3fs" % (end-start))
    else:
        normDF = myDF

Min-max normalize
['_c0', '_c1', '_c2', '_c3']
_c2
_c3
+------+--------+------------------+-------------------+---+
|   _c0|     _c1|               _c2|                _c3|_c4|
+------+--------+------------------+-------------------+---+
|Female|Bulgaria|               0.0|                1.0|  1|
|Female|  Turkey|0.6571428571428571|                0.0|  1|
|  Male|Bulgaria|0.6857142857142857|0.08163265306122448|  0|
|  Male|Bulgaria|0.6857142857142857|0.08163265306122448|  0|
|  Male|   India|0.6285714285714286|0.10204081632653061|  0|
|  Male|Bulgaria|0.7428571428571429|0.12244897959183673|  0|
|  Male| Germany|               1.0|0.05612244897959184|  0|
+------+--------+------------------+-------------------+---+

Time = 4.198s


## Exec: Project 

In [ ]:
    print("Projection")
    all_outputs.append("Projection")
    start = time.time()
    if projection:
        pDF = projectDF(normDF, projdim, isMutable, sc=sc)
    else:
        pDF = normDF
    deltamax = feature_range(pDF)
    print(deltamax)
    # convert into pair rdd
    pDF_pair = pDF.rdd.map(lambda x: (list(x[0:-1]), x[-1]))
    pDF_pair.cache()
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))

Projection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  del sys.path[0]


[0.31622777 0.31622777 0.31622777 0.31622777 0.16537421 0.31622777
 0.31622777 0.15811388 0.         0.15811388 0.15811388 0.31622777
 0.31622777 0.30735403 0.31622777 0.         0.15359634 0.15811388
 0.30332051 0.15811388 0.37931198 0.30735403 0.31622777 0.
 0.15811388 0.15811388 0.15811388 0.15811388 0.30735403 0.        ]
Time = 87.585s


## Exec: Fit

In [ ]:
    print("Fitting")
    all_outputs.append("Fitting")
    start = time.time()
    cf = Chains(deltamax, nchains=nchains, depth=depth, nthreads_fit=nthreads_fit, nthreads_score=nthreads_score, samplerate=samplerate)
    cf = cf.fitparallel(pDF_pair)
    # saving cf
    # pickle.dump(cf, open(model_file, "wb"))
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))

Fitting
[28, 14, 10, 7, 28, 20, 6, 25, 18, 22, 10, 10, 23, 20, 3]
[27, 14, 27, 6, 11, 28, 7, 14, 2, 13, 16, 3, 17, 7, 3][1, 29, 5, 21, 9, 3, 21, 28, 17, 25, 11, 1, 9, 29, 3]
[22, 29, 23, 4, 2, 11, 7, 21, 26, 2, 0, 2, 4, 14, 13][2, 0, 4, 25, 22, 13, 6, 26, 8, 14, 14, 25, 9, 27, 12]

[5, 23, 4, 19, 1, 5, 21, 10, 15, 15, 0, 8, 27, 26, 5]
[19, 23, 21, 23, 0, 23, 19, 10, 16, 7, 3, 5, 7, 19, 29]
[2, 15, 29, 24, 2, 24, 28, 17, 13, 17, 1, 21, 2, 15, 28]

[27, 25, 6, 24, 3, 12, 19, 0, 7, 13, 15, 13, 11, 22, 25]
[4, 25, 20, 29, 22, 8, 11, 20, 0, 25, 25, 0, 14, 1, 29]
[24, 7, 12, 20, 0, 15, 28, 6, 4, 21, 28, 22, 2, 11, 25]
[10, 4, 11, 2, 0, 0, 7, 9, 10, 11, 28, 12, 11, 13, 1]
[16, 29, 19, 15, 24, 21, 12, 26, 18, 16, 3, 11, 28, 8, 18]
[17, 2, 27, 0, 0, 28, 18, 12, 3, 3, 5, 27, 18, 28, 11]
[15, 7, 5, 11, 23, 27, 20, 7, 25, 27, 3, 25, 7, 25, 27]
[12, 25, 21, 9, 17, 18, 4, 14, 24, 1, 9, 23, 29, 17, 26]
[24, 27, 6, 25, 25, 1, 21, 21, 2, 26, 17, 11, 0, 21, 11]
[14, 24, 22, 22, 15, 10, 21, 15, 7, 3, 7, 

## Exec: Score

In [ ]:
    print("Scoring")
    all_outputs.append("Scoring")
    
    anomalyScoresRDD = cf.scoreparallel_test(pDF_pair) #(index, (score, label))
    

Scoring
Score time =  64.4252827167511
Combine time new =  6.289279937744141


## Exec: Metrics

In [ ]:
    scoreslabelsRDD = anomalyScoresRDD.values()
    
    metrics = BinaryClassificationMetrics(scoreslabelsRDD)
    auc = metrics.areaUnderROC
    ap = metrics.areaUnderPR
    
    #end = time.time()
    print("Time = %.3fs" % (end-start))
    #all_outputs.append("Time = %.3fs" % (end-start))
    
    print("xstream: AUC =", auc)
    #all_outputs.append("xstream: AUC = %.5f" % auc)
    
    print("xstream: AP =", ap)
    #all_outputs.append("xstream: AP = %.5f" % ap)

Time = 1556.913s
xstream: AUC = 1.0
xstream: AP = 1.0


## Exec: Explain

In [ ]:
pDFpairKVrdd = pDF_pair.zipWithIndex().map(lambda x:(x[1], x[0])) # (index, (features, label))

topk=3
top = anomalyScoresRDD.takeOrdered(topk, key=lambda x:-x[1][0])
print(top)

[(0, (-2.684620955777741, 1.0)), (1, (-2.836212222467418, 1.0)), (6, (-3.004922588511942, 0.0))]


In [ ]:
xpind = 0 # top anomaly is at index 0

topanomaly = pDFpairKVrdd.filter(lambda x: x[0] == top[xpind][0]).values()
print(pDFpairKVrdd.filter(lambda x: x[0] == top[xpind][0]).collect())

normDFKVrdd = normDF.rdd.zipWithIndex().map(lambda x:(x[1], x[0])) # (index, rawrow)
topORIGanomaly = normDFKVrdd.filter(lambda x: x[0] == top[xpind][0]).values().collect()  
print(list(topORIGanomaly[0])) 

[(0, ([0.3162277660168379, 0.6324555320336758, 0.6324555320336758, 0.0, 0.3162277660168379, 0.6324555320336758, 0.9486832980505137, 0.0, 0.0, 0.0, 0.3162277660168379, 0.6324555320336758, 0.3162277660168379, 0.6324555320336758, 0.0, 0.0, 0.3162277660168379, 0.3162277660168379, 0.6324555320336758, 0.0, 0.3162277660168379, 0.6324555320336758, 0.0, 0.0, 0.3162277660168379, 0.3162277660168379, 0.6324555320336758, 0.3162277660168379, 0.6324555320336758, 0.0], 1))]
['Female', 'Bulgaria', 0.0, 1.0, 1]


In [ ]:
# find scores by in- and out- chains per (projection or original) feature
features = pDF.columns[:-1]

#score from chains that do and do NOT factor f in their anomaly score
fused = np.zeros((len(features), cf.nchains))

score_in_c = np.zeros(len(features))
score_out_c = np.zeros(len(features))

anomaly_score = 0
for cindex in range(0, cf.nchains):
  c = cf.chains[cindex]

  score_c_AllDepths = topanomaly.map(lambda x: c.bincount_score_exp(x[0], cf.chains_cmsketches[cindex]) ).collect()
  #print(score_c_AllDepths)

  ind_min = np.argmin(score_c_AllDepths)
  #print(ind_min)
  if not np.isscalar(ind_min):
    ind_min = ind_min[0]

  score_c = score_c_AllDepths[0][ind_min]
  anomaly_score = anomaly_score + score_c
  
  cSplitFeatures = c.fs
  
  #print(cSplitFeatures[0:ind_min+1])

  for f in range(len(features)):     
    if f in cSplitFeatures[0:ind_min+1]:
      fused[f,cindex] = 1
      score_in_c[f] = score_in_c[f] + score_c
    else:
      score_out_c[f] = score_out_c[f] + score_c

anomaly_score = float(-anomaly_score/cf.nchains)
print(anomaly_score)  
# print(score_in_c)
# print(score_out_c)


-2.684620955777741


In [ ]:
# find importance of each (projection or original) feature

## 1. BY SCORE DIFFERENCE
fimportance = np.zeros(len(features))
for f in range(len(features)):
  num_in = sum(fused[f,:])
  print(num_in.T)
  if num_in != 0:
    if (cf.nchains - num_in) != 0:
      fimportance[f] = float(-score_in_c[f]/num_in) - float(- score_out_c[f]/(cf.nchains - num_in))

print("Importance of (projection) features BY SCORE DIFFERENCE: ")
# the larger and positive, the better
print(fimportance)

## 2. BY "REDUCTION" (may not always be positive?) IN SCORE
fimportance2 = np.zeros(len(features))
for f in range(len(features)):
  num_in = sum(fused[f,:])

  if num_in != 0:
    if (cf.nchains - num_in) != 0:
      fimportance2[f] =  anomaly_score - float(- score_out_c[f]/(cf.nchains - num_in))
# the larger and positive, the better
print("Importance of (projection) features BY REDUCTION IN SCORE")
print(fimportance2)

## 3. addition of those two
fimportance3 = fimportance2 + fimportance
#fimportance3 = fimportance3 / fimportance3.sum() !!!! IF THE SUM IS NEGATIVE, TROUBLE
print(fimportance3)


3.0
2.0
4.0
0.0
4.0
2.0
1.0
1.0
1.0
1.0
2.0
0.0
3.0
2.0
3.0
3.0
3.0
2.0
2.0
2.0
1.0
1.0
3.0
0.0
4.0
1.0
0.0
3.0
2.0
2.0
Importance of (projection) features BY SCORE DIFFERENCE: 
[-0.39576671  0.71314683  0.74415321  0.         -0.34280331  0.71314683
  0.69859281 -0.91871586 -1.34222351 -0.32181535 -0.63318781  0.
  0.72832017  0.71314683 -0.04115679 -1.04472948 -0.39576671  0.71314683
 -0.49619072  0.71314683  0.1016923   0.69859281  0.72832017  0.
 -1.22036301 -0.32181535  0.         -0.33550962  0.1923135  -1.15402114]
Importance of (projection) features BY REDUCTION IN SCORE
[-0.023746    0.02852587  0.05953226  0.         -0.02742426  0.02852587
  0.01397186 -0.01837432 -0.02684447 -0.00643631 -0.02532751  0.
  0.04369921  0.02852587 -0.00246941 -0.06268377 -0.023746    0.02852587
 -0.01984763  0.02852587  0.00203385  0.01397186  0.04369921  0.
 -0.09762904 -0.00643631  0.         -0.02013058  0.00769254 -0.04616085]
[-0.41951272  0.7416727   0.80368547  0.         -0.37022757  0.

In [ ]:
### R_ohe IS THE SAME FOR ALL EXPLANATIONS: BUILD ONCE ONLY
if projection: ## ----------------> moved to explain_transform_ohe
    feature_names = all_columns
    ndim = len(feature_names)

    X = normDF.take(1)
    #print(X)
    types = [type(X[0][i]) == str for i in range(ndim)]
    #print(types)
    # find total # of features after OHE
    num_unique = np.zeros((ndim,1))
    for i in range(ndim):
        if types[i]:
          num_unique[i] = normDF.select(normDF.columns[i]).distinct().count()
        else:
          num_unique[i] = 1

    #print(num_unique)

    totalAfterOHE = num_unique.sum()
    
    R_ohe = np.zeros((projdim, int(totalAfterOHE))) 


In [ ]:
### R_ohe IS THE SAME FOR ALL EXPLANATIONS: BUILD ONCE ONLY

#i=1  ## ----------------> moved to explain_transform_ohe
# distinctDF = normDF.select(normDF.columns[i]).distinct()
#dvals = distinctDF.rdd.collect()
#print(dvals[0][0])
#outrdd = distinctDF.rdd.map(lambda x: (x[0], np.array([_hash_string(k, x[0]) for k in keys])))
#print(outrdd.collect())
if projection:
  def _hash_string(k, s):
      hash_value = int(mmh3.hash(s, signed=False, seed=k))/(2.0**32-1)
      den = 1/3
      if hash_value <= den/2.0:
          return 1 #-1 
      elif hash_value <= den:
          return 1
      else:
          return 0


  random.seed(42)
  keys = np.arange(0, projdim, 1)
  ind = 0
  for i in range(ndim): # for each original feature
      if types[i]:
          # find all distinct values of an original categorical feature:
          distinctDF = normDF.select(normDF.columns[i]).distinct()
          # hash each unique name.value combination K times
          outrdd = distinctDF.rdd.map(lambda x: (x[0], np.array([_hash_string(k, feature_names[i]+'.'+x[0]) for k in keys])))
          for v in outrdd.collect():
              print(v[0])
              #print(abs(v[1]))
              R_ohe[:,ind] = v[1].T #abs(v[1]).T
              #print(abs(v[1]).sum())
              ind = ind+1
      else:
        f = feature_names[i]
        R_ohe[:,ind] = np.array([_hash_string(k, f) for k in keys]) #abs(np.array([_hash_string(k, f) for k in keys]))
        ind = ind+1 # increment by 1 as just 1 ohe feature

  np.set_printoptions(threshold=np.inf)
  np.set_printoptions(linewidth=200)
  print(R_ohe)

Female
Male
Turkey
Germany
India
Bulgaria
[[1. 0. 1. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 1. 1. 0. 0.]
 [1. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 1. 0. 0. 1.]
 [1. 0. 0. 0. 0. 1. 0. 0.]
 [1. 1. 0. 0. 1. 1. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]]


In [ ]:
#if projection:
  
  #feature_names = all_columns
  #print(feature_names)

  # projector = StreamhashProjection(n_components=projdim, 
  #                                   feature_names=feature_names,
  #                                   density=1/3.0, 
  #                                   random_state=42, 
  #                                   mutable=True,
  #                                   sc=sc)

  # hashR = projector.explain_transform_ohe(normDF, feature_names)

  # #### newR is a KXd binary matrix, rij=1 means projection i contains feature j
  # newR = np.abs(hashR) / projector.constant
  # print(newR)


In [ ]:
# attribution to original features
# 1. (Naive) BY AVG PROJECTION-FEATURE IMPORTANCES
if projection: 
  fimpr = fimportance3.copy() # which one to use

  totalprojperfeat = np.sum(R_ohe, axis=0)
  print(totalprojperfeat)
  totalfeatureperproj = np.sum(R_ohe, axis=1)
  print(totalfeatureperproj)

  print(fimpr)
  scaledR = R_ohe.copy()
  for p in range(len(features)):
      scaledR[p,:] = scaledR[p,:] * fimpr[p]

  np.set_printoptions(threshold=np.inf)
  np.set_printoptions(linewidth=200)
  print(scaledR)
  #print(scaledR.T)
  origfimportance = np.sum(scaledR, axis=0) / totalprojperfeat

  print(origfimportance)

[5. 2. 3. 1. 5. 4. 1. 2.]
[3. 3. 3. 2. 3. 2. 5. 1. 0. 1.]
[-0.42140267  0.81855447  0.72272923  0.82435275 -0.4681285  -0.30333482  0.         -0.95090915 -1.12106027 -0.21242877]
[[-0.42140267 -0.         -0.42140267 -0.         -0.42140267 -0.         -0.         -0.        ]
 [ 0.81855447  0.          0.          0.          0.81855447  0.81855447  0.          0.        ]
 [ 0.72272923  0.          0.          0.          0.72272923  0.72272923  0.          0.        ]
 [ 0.          0.          0.          0.82435275  0.          0.          0.82435275  0.        ]
 [-0.         -0.         -0.4681285  -0.         -0.4681285  -0.         -0.         -0.4681285 ]
 [-0.30333482 -0.         -0.         -0.         -0.         -0.30333482 -0.         -0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.          0.        ]
 [-0.         -0.         -0.95090915 -0.         -0.         -0.         -0.         -0.        ]
 [-0.         -0.         -0

In [ ]:
# attribution to original features
# 1. BY RANDOM WALKS

dr = np.random.rand(R_ohe.shape[1],1)
pr = np.random.rand(R_ohe.shape[0],1)


sumr = dr.sum()+pr.sum()
dr = dr / sumr
pr = pr / sumr
print("Number of original features ", R_ohe.shape[1])
print("Number of projected features ", R_ohe.shape[0])
alpha = 0.15

fimpr = fimportance3.copy() # which one to use
print(fimpr)
print(fimpr.sum())

newR = R_ohe.copy()

#Option 1: # Ignore negative (projection) features
indices = np.array(np.where(fimpr < 0))
          #Remove associated edges
print(indices)
newR[indices[0],:] =0 
print(newR.T)

fimpr[fimpr < 0] = 0 

#Option 2: # Shift so that minimum is zero
#if np.min(fimpr) < 0:
#  fimpr = fimpr + np.abs(np.min(fimpr))  

print(fimpr) 
print(fimpr.sum())
fimpr = fimpr / sum(fimpr) # normalize so sum to 1, a prob dist.n

fimpr = fimpr.reshape(newR.shape[0],1)
#print(fimpr)
print(fimpr.sum())
print(dr.sum()+pr.sum())

sums = newR.sum(axis=0,keepdims=1)
#print(sums)
sums[sums==0] = 1
A = newR.copy() / sums
#print(A)

B = newR.copy().T
sums = B.sum(axis=0,keepdims=1)
#print(sums)
sums[sums==0] = 1
B = B / sums
print(B)

for i in range(1,100):
    drnew = alpha * np.dot(B, pr) # + (1-alpha) * all_zeros ### UPDATED IMPORTANCE OF ORIG FEATURES
    prnew = alpha * np.dot(A, dr) + (1-alpha) * fimpr       ### UPDATED IMPORTANCE OF PROJ FEATURES
    denom = (drnew.sum()+prnew.sum())
    #print(denom) # should be 1
    dr = drnew.reshape(R_ohe.shape[1],1) / denom
    pr = prnew.reshape(R_ohe.shape[0],1) / denom
    #print()

print(dr/dr.sum())



Number of original features  8
Number of projected features  10
[ 0.69958179 -0.72608848 -0.59186913 -0.29578369 -0.10775036 -0.29170927  0.34287704  0.91669548 -1.13578369  0.06929278]
-1.1205375288621222
[[1 2 3 4 5 8]]
[[1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
[0.69958179 0.         0.         0.         0.         0.         0.34287704 0.91669548 0.         0.06929278]
2.0284470917293373
1.0
1.0
[[0.33333333 0.         0.         0.         0.         0.         0.2        0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.         0.2        0.         0.         1.        ]
 [0.33333333 0.         0.         0.         0.         0.         0.         1.         0.         0.        ]
 [0.         0.         0.         

In [ ]:
# attribution to original features
# 2. BY HITS
from numpy import linalg as LA

print("Number of original features ", R_ohe.shape[1])
print("Number of projected features ", R_ohe.shape[0])
alpha = .5

newR = R_ohe.copy()

#Option 0: Use with BOTH + & -
fimpr = fimportance3.copy() # which one to use

#Option 1: # Ignore negative-importance (projection) features
#indices = np.array(np.where(fimpr < 0))
          #Remove associated edges
#print(indices)
#newR[indices[0],:] =0 
#print(newR.T)
#fimpr[fimpr < 0] = 0 

#Option 2: # Shift so that minimum is zero
#if np.min(fimpr) < 0:
#  fimpr = fimpr + np.abs(np.min(fimpr))   


#fimpr = fimpr / LA.norm(fimpr,2)

fimpr = fimpr.reshape(R_ohe.shape[0],1)
pr = fimpr.copy()
print(pr)
#print(fimpr.sum())

B = newR.T
sums = B.sum(axis=0,keepdims=1)
#print(sums)
sums[sums==0] = 1
B = B / sums
print(B)

for i in range(1,3):
    #dr = np.dot(R_ohe.T, pr)
    dr = np.dot(B, pr)
    ## normalize dr:
    #print(dr.sum())
    dr = dr.reshape(newR.shape[1],1) / LA.norm(dr,2) #dr.sum()
    #print(dr)
    pr = np.dot(newR, dr)
    ## normalize pr:
    #print(pr.sum())
    
    pr = alpha * pr + (1-alpha) * fimpr
    pr = pr.reshape(newR.shape[0],1) / LA.norm(pr,2) #pr.sum()  
    #print(fimpr)
     
print(dr)


Number of original features  8
Number of projected features  10
[[ 0.69958179]
 [-0.72608848]
 [-0.59186913]
 [-0.29578369]
 [-0.10775036]
 [-0.29170927]
 [ 0.34287704]
 [ 0.91669548]
 [-1.13578369]
 [ 0.06929278]]
[[0.33333333 0.33333333 0.33333333 0.         0.         0.5        0.2        0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.         0.2        0.         0.         1.        ]
 [0.33333333 0.         0.         0.         0.33333333 0.         0.         1.         0.         0.        ]
 [0.         0.         0.         0.5        0.         0.         0.         0.         0.         0.        ]
 [0.33333333 0.33333333 0.33333333 0.         0.33333333 0.         0.2        0.         0.         0.        ]
 [0.         0.33333333 0.33333333 0.         0.         0.5        0.2        0.         0.         0.        ]
 [0.         0.         0.         0.5        0.         0.         0.         0.         0.         0.    

In [ ]:
# attribution to original features
# 3. BY SIGNED HITS

epsilon = 0.1

dr = np.ones(R_ohe.shape[1]) * epsilon # importance
pr = np.ones(R_ohe.shape[0]) * epsilon # authority/accurateness
#print(pr)

B = R_ohe.copy().T
sums = B.sum(axis=0,keepdims=1)
#print(sums)
sums[sums==0] = 1
scaledRtrans = scaledR.T / sums
print(scaledRtrans)

for i in range(1,20):
    #dr = np.dot(R_ohe.T, pr)
    dr = np.dot(scaledRtrans, pr)
    #newdr = np.array([1 - (1 / (1 + np.exp(x)))  for x in dr])
    #dr = dr + np.abs(np.min(dr))  
    #dr = np.array(alpha * dr.T + (1-alpha) * np.ones(R_ohe.shape[1]) / R_ohe.shape[1])
    ## normalize dr:
    dr = dr.reshape(scaledR.shape[1],1) / LA.norm(dr,2) # abs(dr.sum())
    #print(dr)

    pr = np.dot(scaledRtrans.T, dr)
    ## normalize pr:
    #print(pr.sum())
    #pr = pr + np.abs(np.min(pr))  
    
    pr = pr.reshape(scaledR.shape[0],1) / LA.norm(pr,2) # abs(pr.sum())
    #print(fimpr)
     
print(dr)


[[ 0.23319393 -0.24202949 -0.19728971 -0.         -0.         -0.14585463  0.06857541  0.         -0.          0.        ]
 [ 0.         -0.         -0.         -0.         -0.         -0.          0.06857541  0.         -0.          0.06929278]
 [ 0.23319393 -0.         -0.         -0.         -0.03591679 -0.          0.          0.91669548 -0.          0.        ]
 [ 0.         -0.         -0.         -0.14789184 -0.         -0.          0.          0.         -0.          0.        ]
 [ 0.23319393 -0.24202949 -0.19728971 -0.         -0.03591679 -0.          0.06857541  0.         -0.          0.        ]
 [ 0.         -0.24202949 -0.19728971 -0.         -0.         -0.14585463  0.06857541  0.         -0.          0.        ]
 [ 0.         -0.         -0.         -0.14789184 -0.         -0.          0.          0.         -0.          0.        ]
 [ 0.         -0.         -0.         -0.         -0.03591679 -0.          0.06857541  0.         -0.          0.        ]]
[[ 9.97925047e-

## Running options

In [ ]:
def run_xstream_all():
    all_outputs = []
    min_max = True
    if min_max:
        print("Min-max normalize")
        all_outputs.append("Min-max normalize")
        start = time.time()
        normDF = min_max_normalize(myDF) #100+ columns
        normDF.show()
        end = time.time()
        print("Time = %.3fs" % (end-start))
        all_outputs.append("Time = %.3fs" % (end-start))
    else:
        normDF = myDF
    
    print("Projection")
    all_outputs.append("Projection")
    start = time.time()
    if projection:
        pDF = projectDF(normDF, projdim, sc=sc)
    else:
        pDF = normDF
    deltamax = feature_range(pDF)
    print(deltamax)
    # convert into pair rdd
    pDF_pair = pDF.rdd.map(lambda x: (list(x[0:-1]), x[-1]))
    pDF_pair.cache()
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))
    
    print("Fitting")
    all_outputs.append("Fitting")
    start = time.time()
    cf = Chains(deltamax, nchains=nchains, depth=depth, nthreads_fit=nthreads_fit, nthreads_score=nthreads_score, samplerate=samplerate)
    cf = cf.fitparallel(pDF_pair)
    # saving cf
    # pickle.dump(cf, open(model_file, "wb"))
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))
    
    print("Scoring")
    all_outputs.append("Scoring")
    start = time.time()
    #anomalyScoresRDD = pDF_pair.map(lambda x: (float(cf.score(x[0])), float(x[1])))
    anomalyScoresRDD = cf.scoreparallel_test(pDF_pair)
    
    metrics = BinaryClassificationMetrics(anomalyScoresRDD)
    auc = metrics.areaUnderROC
    ap = metrics.areaUnderPR
    
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))
    
    print("xstream: AUC =", auc)
    all_outputs.append("xstream: AUC = %.5f" % auc)
    
    print("xstream: AP =", ap)
    all_outputs.append("xstream: AP = %.5f" % ap)
    
    print("Printing")
    all_outputs.append("Printing")
    start = time.time()
    
    anomalyScoresRDD.toDF().repartition(1).write.format('com.databricks.spark.csv').save("/ocean/projects/cie170025p/shared/SpamURL_scores/" + score_folder, header = 'false')
    
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))
    
    with open(output_file, 'w') as f:
        for s in all_outputs:
            print(s, file=f)

def run_xstream_fit():
    all_outputs = []
    min_max = False
    if min_max:
        print("Min-max normalize")
        all_outputs.append("Min-max normalize")
        start = time.time()
        normDF = min_max_normalize(myDF) #100+ columns
        normDF.show()
        end = time.time()
        print("Time = %.3fs" % (end-start))
        all_outputs.append("Time = %.3fs" % (end-start))
    else:
        normDF = myDF
    
    print("Projection")
    all_outputs.append("Projection")
    start = time.time()
    if projection:
        pDF = projectDF(normDF, projdim, sc=sc)
    else:
        pDF = normDF
    deltamax = feature_range(pDF)
    print(deltamax)
    # convert into pair rdd
    pDF_pair = pDF.rdd.map(lambda x: (list(x[0:-1]), x[-1]))
    pDF_pair.cache()
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))
    
    print("Fitting")
    all_outputs.append("Fitting")
    start = time.time()
    cf = Chains(deltamax, nchains=nchains, depth=depth, nthreads_fit=nthreads_fit, nthreads_score=nthreads_score, samplerate=samplerate)
    cf = cf.fitparallel(pDF_pair)
    # saving cf
    pickle.dump(cf, open(model_file, "wb"))
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))
    
    with open(output_file, 'w') as f:
        for s in all_outputs:
            print(s, file=f)

def run_xstream_score_print():
    all_outputs = []
    min_max = False
    if min_max:
        print("Min-max normalize")
        all_outputs.append("Min-max normalize")
        start = time.time()
        normDF = min_max_normalize(myDF) #100+ columns
        normDF.show()
        end = time.time()
        print("Time = %.3fs" % (end-start))
        all_outputs.append("Time = %.3fs" % (end-start))
    else:
        normDF = myDF
    
    print("Projection")
    all_outputs.append("Projection")
    start = time.time()
    if projection:
        pDF = projectDF(normDF, projdim, sc=sc)
    else:
        pDF = normDF
    deltamax = feature_range(pDF)
    print(deltamax)
    # convert into pair rdd
    pDF_pair = pDF.rdd.map(lambda x: (list(x[0:-1]), x[-1]))
    pDF_pair.cache()
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))
    
    cf = pickle.load(open(model_file, "rb"))
    print("Scoring")
    all_outputs.append("Scoring")
    start = time.time()
    #anomalyScoresRDD = pDF_pair.map(lambda x: (float(cf.score(x[0])), float(x[1])))
    anomalyScoresRDD = cf.scoreparallel_test(pDF_pair)    
    
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))
    
    print("Printing")
    all_outputs.append("Printing")
    start = time.time()
    
    anomalyScoresRDD.toDF().repartition(1).write.format('com.databricks.spark.csv').save("/ocean/projects/cie170025p/shared/" + score_folder, header = 'false')
    #pickle.dump(anomalyScores, open("/ocean/projects/cie170025p/shared/score_" + str(part) + ".pkl", "wb"))
    
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))

    with open(output_file, 'w') as f:
        for s in all_outputs:
            print(s, file=f)
            
def run_xstream_score_all():
    all_outputs = []
    min_max = False
    if min_max:
        print("Min-max normalize")
        all_outputs.append("Min-max normalize")
        start = time.time()
        normDF = min_max_normalize(myDF) #100+ columns
        normDF.show()
        end = time.time()
        print("Time = %.3fs" % (end-start))
        all_outputs.append("Time = %.3fs" % (end-start))
    else:
        normDF = myDF
    
    print("Projection")
    all_outputs.append("Projection")
    start = time.time()
    if projection:
        pDF = projectDF(normDF, projdim, sc=sc)
    else:
        pDF = normDF
    deltamax = feature_range(pDF)
    print(deltamax)
    # convert into pair rdd
    pDF_pair = pDF.rdd.map(lambda x: (list(x[0:-1]), x[-1]))
    pDF_pair.cache()
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))
    
    cf = pickle.load(open(model_file, "rb"))
    print("Scoring")
    all_outputs.append("Scoring")
    start = time.time()
    #anomalyScoresRDD = pDF_pair.map(lambda x: (float(cf.score(x[0])), float(x[1])))
    anomalyScoresRDD = cf.scoreparallel_test(pDF_pair)
    
    metrics = BinaryClassificationMetrics(anomalyScoresRDD)
    auc = metrics.areaUnderROC
    ap = metrics.areaUnderPR
    
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))
    
    print("xstream: AUC =", auc)
    all_outputs.append("xstream: AUC = %.5f" % auc)
    
    print("xstream: AP =", ap)
    all_outputs.append("xstream: AP = %.5f" % ap)
    
def run_xstream_score_combine():
    all_outputs = []
        
    print("Loading")
    all_outputs.append("Loading")
    start = time.time()

    score_folder = "score_" + str(nchains) + "_" + str(depth) + "_1.csv"
    scores_1 = pd.read_csv("/ocean/projects/cie170025p/shared/" + score_folder + "/score.csv", header=None)
    score_folder = "score_" + str(nchains) + "_" + str(depth) + "_2.csv"
    scores_2 = pd.read_csv("/ocean/projects/cie170025p/shared/" + score_folder + "/score.csv", header=None)
    score_folder = "score_" + str(nchains) + "_" + str(depth) + "_3.csv"
    scores_3 = pd.read_csv("/ocean/projects/cie170025p/shared/" + score_folder + "/score.csv", header=None)
    score_folder = "score_" + str(nchains) + "_" + str(depth) + "_4.csv"
    scores_4 = pd.read_csv("/ocean/projects/cie170025p/shared/" + score_folder + "/score.csv", header=None)
    
    all_scores = pd.concat([scores_1, scores_2, scores_3, scores_4])
    
    del scores_1
    del scores_2
    del scores_3
    del scores_4
    
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))
    
    print("Scoring")
    all_outputs.append("Scoring")
    start = time.time()
    
    y_score = all_scores[0].to_numpy()
    y_true = all_scores[1].to_numpy()
    
    auc = roc_auc_score(y_true, y_score)
    ap = average_precision_score(y_true, y_score)
    
    print("xstream: AUC =", auc)
    all_outputs.append("xstream: AUC = %.5f" % auc)
    
    print("xstream: AP =", ap)
    all_outputs.append("xstream: AP = %.5f" % ap)
    
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))
    
    print("Scoring F1")
    all_outputs.append("Scoring F1")
    start = time.time()
    
    all_scores.sort_values(by=[0], inplace=True, ascending=False)

    num_outliers = 1000000
    num_selected = 1000000 # 500000, 1000000, 2000000
    tp = 0
    fp = 0
    for i in range(num_selected):
        if all_scores.iloc[i, 1] == 1:
            tp += 1
        else:
            fp += 1

    precision = tp / (tp + fp)
    recall = tp / num_outliers
    f1 = 2 * precision * recall / (precision + recall)
    
    print("xstream: F1 =", f1)
    all_outputs.append("xstream: F1 = %.5f" % f1)
    
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))
    
    with open(output_file, 'w') as f:
        for s in all_outputs:
            print(s, file=f)
            
def run_xstream_score_combine_10():
    all_outputs = []
        
    print("Loading")
    all_outputs.append("Loading")
    start = time.time()

    score_folder = "score_" + str(nchains) + "_" + str(depth) + "_1.csv"
    scores_1 = pd.read_csv("/ocean/projects/cie170025p/shared/" + score_folder + "/score.csv", header=None)
    score_folder = "score_" + str(nchains) + "_" + str(depth) + "_2.csv"
    scores_2 = pd.read_csv("/ocean/projects/cie170025p/shared/" + score_folder + "/score.csv", header=None)
    score_folder = "score_" + str(nchains) + "_" + str(depth) + "_3.csv"
    scores_3 = pd.read_csv("/ocean/projects/cie170025p/shared/" + score_folder + "/score.csv", header=None)
    score_folder = "score_" + str(nchains) + "_" + str(depth) + "_4.csv"
    scores_4 = pd.read_csv("/ocean/projects/cie170025p/shared/" + score_folder + "/score.csv", header=None)
    score_folder = "score_" + str(nchains) + "_" + str(depth) + "_5.csv"
    scores_5 = pd.read_csv("/ocean/projects/cie170025p/shared/" + score_folder + "/score.csv", header=None)
    score_folder = "score_" + str(nchains) + "_" + str(depth) + "_6.csv"
    scores_6 = pd.read_csv("/ocean/projects/cie170025p/shared/" + score_folder + "/score.csv", header=None)
    score_folder = "score_" + str(nchains) + "_" + str(depth) + "_7.csv"
    scores_7 = pd.read_csv("/ocean/projects/cie170025p/shared/" + score_folder + "/score.csv", header=None)
    score_folder = "score_" + str(nchains) + "_" + str(depth) + "_8.csv"
    scores_8 = pd.read_csv("/ocean/projects/cie170025p/shared/" + score_folder + "/score.csv", header=None)
    score_folder = "score_" + str(nchains) + "_" + str(depth) + "_9.csv"
    scores_9 = pd.read_csv("/ocean/projects/cie170025p/shared/" + score_folder + "/score.csv", header=None)
    score_folder = "score_" + str(nchains) + "_" + str(depth) + "_10.csv"
    scores_10 = pd.read_csv("/ocean/projects/cie170025p/shared/" + score_folder + "/score.csv", header=None)
    
    all_scores = pd.concat([scores_1, scores_2, scores_3, scores_4, scores_5, scores_6, scores_7, scores_8, scores_9, scores_10])
    
    del scores_1
    del scores_2
    del scores_3
    del scores_4
    del scores_5
    del scores_6
    del scores_7
    del scores_8
    del scores_9
    del scores_10
    
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))
    
    print("Scoring")
    all_outputs.append("Scoring")
    start = time.time()
    
    y_score = all_scores[0].to_numpy()
    y_true = all_scores[1].to_numpy()
    
    auc = roc_auc_score(y_true, y_score)
    ap = average_precision_score(y_true, y_score)
    
    print("xstream: AUC =", auc)
    all_outputs.append("xstream: AUC = %.5f" % auc)
    
    print("xstream: AP =", ap)
    all_outputs.append("xstream: AP = %.5f" % ap)
    
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))
    
    print("Scoring F1")
    all_outputs.append("Scoring F1")
    start = time.time()
    
    all_scores.sort_values(by=[0], inplace=True, ascending=False)

    num_outliers = 1000000
    num_selected = 1000000 # 500000, 1000000, 2000000
    tp = 0
    fp = 0
    for i in range(num_selected):
        if all_scores.iloc[i, 1] == 1:
            tp += 1
        else:
            fp += 1

    precision = tp / (tp + fp)
    recall = tp / num_outliers
    f1 = 2 * precision * recall / (precision + recall)
    
    print("xstream: F1 =", f1)
    all_outputs.append("xstream: F1 = %.5f" % f1)
    
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))

    with open(output_file, 'w') as f:
        for s in all_outputs:
            print(s, file=f)
            
def run_xstream_score_f1():
    all_outputs = []
        
    print("Loading")
    all_outputs.append("Loading")
    start = time.time()

    all_scores = pd.read_csv("/ocean/projects/cie170025p/shared/SpamURL_scores/" + score_folder + "/score.csv", header=None)
    
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))
    
    print("Scoring F1")
    all_outputs.append("Scoring F1")
    start = time.time()
    
    all_scores.sort_values(by=[0], inplace=True, ascending=False)

    num_outliers = 792145
    num_selected = 792145
    tp = 0
    fp = 0
    for i in range(num_selected):
        if all_scores.iloc[i, 1] == 1:
            tp += 1
        else:
            fp += 1

    precision = tp / (tp + fp)
    recall = tp / num_outliers
    f1 = 2 * precision * recall / (precision + recall)
    
    print("xstream: F1 =", f1)
    all_outputs.append("xstream: F1 = %.5f" % f1)
    
    end = time.time()
    print("Time = %.3fs" % (end-start))
    all_outputs.append("Time = %.3fs" % (end-start))
    
    with open(output_file, 'w') as f:
        for s in all_outputs:
            print(s, file=f)

In [ ]:
#from memory_profiler import memory_usage
memory = False
if memory:
    print("Testing memory usage")
    usage = memory_usage(proc=run_xstream_all)
    #usage = memory_usage(proc=run_xstream_score_f1)
    #usage = memory_usage(proc=run_xstream_fit)
    #usage = memory_usage(proc=run_xstream_score_print)
    #usage = memory_usage(proc=run_xstream_score_combine_10)
    print("Peak memory = %.3fM" % max(usage))
    print("Average memory = %.3fM" % (sum(usage)/len(usage)))
    with open(output_file, 'a') as f:
        print("Peak memory = %.3fM" % max(usage), file=f)
        print("Average memory = %.3fM" % (sum(usage)/len(usage)), file=f)
else:
    #run_xstream_score_combine()
    run_xstream_all()

Projection


Py4JJavaError: ignored